In [8]:
import os
import dgl
import time
import random
import argparse
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.sampling import select_topk
from dgl import function as fn
from dgl.nn.functional import edge_softmax
from utils import accuracy, augmentation
from model import GCN, APPNP, GAT, GConv, MLP

# from data import load_data
from data_jd import load_data
from data_loader import data_loader
from tools import evaluate_results_nc


'''
seed = 9
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
'''

# GCN
# cora 0.01 1e-3 500 32 0.7 1.0 0.5
# cite 0.01 1e-3 500 32 0.1 1.0 0.5
# pubm 0.01 1e-3 500 32 0.5 0.5 0.2
# comp 0.01 1e-3 500 32 0.3 0.7 0.5
# phot 0.01 1e-3 500 32 0.5 0.5 0.5

# less label
# GCN
# cora 0.01 1e-3 500 32 0.8 1.0 0.7 
# cite 0.01 1e-3 500 32 0.8 1.0 1.1
# pubm 0.01 1e-3 500 32 0.5 0.5 0.5


device = 'cuda:0'

parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default='JD2')
parser.add_argument('--lr', type=float, default=0.01, help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=1e-3, help='Weight decay')
parser.add_argument('--epochs', type=int, default=500, help='Number of epochs to train.')
parser.add_argument('--hidden', type=int, default=4, help='Number of hidden units.')
parser.add_argument('--dropout', type=float, default=0.5, help='Dropout rate')
parser.add_argument('--alpha', type=float, default=1, help='loss hyperparameter')
parser.add_argument('--temp', type=float, default=0.5, help='sharpen temperature')
parser.add_argument('--prob', type=float, default=0.5, help='loss hyperparameter')
parser.add_argument('--seed', type=int, default=0, help='Patience')
parser.add_argument('--feats-type', type=int, default=3,
                        help='Type of the node features used. ' +
                        '0 - loaded features; ' +
                        '1 - only target node features (zero vec for others); ' +
                        '2 - only target node features (id vec for others); ' +
                        '3 - all id vec. Default is 2;' +
                        '4 - only term features (id vec for others);' +
                        '5 - only term features (zero vec for others).')
args = parser.parse_args(args=[])

print('loading data >>>>>>>>>>>>>>>>>>>>>')
dataset = data_loader('/ossfs/workspace/NASA-main/data/'+args.dataset)
edge_dict = {}

ii = 0
for i, meta_path in dataset.links['meta'].items():
    edge_dict[(str(meta_path[0]), str(meta_path[0]) + '_edge' + str(i) + '_' + str(meta_path[1]), str(meta_path[1]))] = (torch.tensor(dataset.links['data'][i].tocoo().row - dataset.nodes['shift'][meta_path[0]]), torch.tensor(dataset.links['data'][i].tocoo().col - dataset.nodes['shift'][meta_path[1]]))
#         ii += 1
#         edge_dict[(str(meta_path[1]), str(meta_path[1]) + '_edge' + str(ii) + '_' + str(meta_path[0]), str(meta_path[0]))] = (torch.tensor(dataset.links['data'][i].tocoo().col - dataset.nodes['shift'][meta_path[1]]), torch.tensor(dataset.links['data'][i].tocoo().row - dataset.nodes['shift'][meta_path[0]]))
#         ii += 1
       
        
node_count = {}
for i, count in dataset.nodes['count'].items():
    node_count[str(i)] = count
    print(i, node_count)

print('building dgl.heterograph >>>>>>>>>>>>>>>>>>>>>')
G = dgl.heterograph(edge_dict, num_nodes_dict = node_count, device=device)  # , idtype=torch.int64

G.node_dict = {}
G.edge_dict = {}
for ntype in G.ntypes:
    G.node_dict[ntype] = len(G.node_dict)
for etype in G.etypes:
    G.edge_dict[etype] = len(G.edge_dict)
    G.edges[etype].data['id'] = torch.ones(G.number_of_edges(etype), dtype=torch.long).to(device) * G.edge_dict[etype] 
    print('number_of_edges>>>>>>>>',etype,'**',G.number_of_edges(etype))

feats_type = args.feats_type
features_list, adjM, labels, train_val_test_idx, dl = load_data(args.dataset)
print('features_list0>>>>>>>>',features_list[0].shape)

## 添加图特征
for ntype in G.ntypes:
    G.nodes[ntype].data['feats'] = torch.from_numpy(features_list[int(ntype)]).to(torch.float32).to(device)

# calculate norm for each edge type and store in edge
for canonical_etype in G.canonical_etypes:
    u, v, eid = G.all_edges(form='all', etype=canonical_etype)
    _, inverse_index, count = torch.unique(
        v, return_inverse=True, return_counts=True)
    degrees = count[inverse_index]
    norm = torch.ones(eid.shape[0]).float().to(device) / degrees.float().to(device)
    norm = norm.unsqueeze(1)
    G.edges[canonical_etype].data['norm'] = norm

# retrieve labels of ground truth
labels = torch.LongTensor(labels).to(device)
train_idx = train_val_test_idx['train_idx']
train = np.sort(train_idx)
val_idx = train_val_test_idx['val_idx']
val = np.sort(val_idx)
test_idx = train_val_test_idx['test_idx']
test = np.sort(test_idx)

print(len(train_idx), len(test_idx), len(val_idx))

g = dgl.to_homogeneous(G, ndata=['feats'], edata=['norm'])
g = dgl.add_self_loop(g)


# retrieve the number of classes
nclass = dl.labels_train['num_classes']
print('nclass>>>>>>>>>>>',nclass)

augmentor = augmentation(g, g.ndata['feats'])
augmentor.init()

g = g.to(device)
# Extract node features
feats = g.ndata.pop('feats').to(device)


net = GCN(feats.size()[1], args.hidden, nclass, args.dropout).to(device)

# create optimizer
optimizer = torch.optim.AdamW(net.parameters(), lr=args.lr, weight_decay=args.weight_decay)

# main loop
dur = []
log = []
counter = 0

bce_loss = nn.BCELoss(reduction='mean')
mce_loss = nn.NLLLoss(reduction='mean')
mse_loss = nn.MSELoss(reduction='mean')
kld_loss = nn.KLDivLoss(reduction='batchmean')


def edge_kld_loss(edges):
    src = edges.src['pred'] + 1e-10
    dst = (edges.dst['tar'] + 1e-10).detach()
    loss = (dst*dst.log10() - dst*src.log()).sum(1, keepdim=True)
    return {'loss': loss}


def label_prop_loss(edges):
    src = edges.src['pred'] + 1e-5
    dst = (edges.dst['pred'] + 1e-5).detach()
    loss = (dst*dst.log() - dst*src.log()).sum(1, keepdim=True)
    return {'loss': loss}


def evaluate(model_pred, labels):
    # 注意这里的model_pred是经过sigmoid处理的，sigmoid处理后可以视为预测是这一类的概率
    # 预测结果，大于这个阈值则视为预测正确
    pred_result = model_pred.argmax(dim=1)
    labels = labels.cpu()
    pred_result = pred_result.cpu()

    micro = f1_score(labels, pred_result, average='micro')
    macro = f1_score(labels, pred_result, average='macro')

    return micro, macro

acc_best = 0

for epoch in range(args.epochs):
    t0 = time.time()

    aug_g, aug_feat = augmentor.generate(strategy='replace', ratio=args.prob)
    aug_g = aug_g.to(device)
    aug_feat = aug_feat.to(device)

    net.train()
    aug_pred, _ = net(aug_g, aug_feat)
    loss1 = mce_loss((aug_pred + 1e-1).log()[train], labels[train])
#     print('aug_pred>>>>>>>>>>>',aug_pred)
#     print('labels>>>>>>>>>>>',labels.shape)

    aug_g.ndata['pred'] = aug_pred
    aug_g.update_all(fn.copy_u('pred', '_'), fn.mean('_', 'avg_pred'))
    avg_pred = aug_g.ndata.pop('avg_pred')

    sharp = (torch.pow(avg_pred, 1./args.temp) / torch.sum(torch.pow(avg_pred, 1./args.temp), dim=1, keepdim=True))
    aug_g.ndata['tar'] = sharp

    aug_g.update_all(label_prop_loss, fn.mean('loss', 'loss'))
    loss2 = aug_g.ndata.pop('loss').mean()

#     print('loss2>>>>>>>>>>',loss2)
    loss = loss1 + args.alpha * loss2
    print('loss1，loss2>>>>>>>>>>',loss1,loss2)
    print('loss>>>>>>>>>>',loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
#     for name, parms in net.named_parameters():
#         print('-->name:', name, '-->grad_requirs:', parms.requires_grad, '--weight', torch.mean(parms.data), ' -->grad_value:', torch.mean(parms.grad))

    net.eval()
    with torch.no_grad():
        pred, _ = net(g, feats)

        train_loss = F.nll_loss((pred[train] + 1e-10).log10(), labels[train])
        val_loss = F.nll_loss((pred[val] + 1e-10).log10(), labels[val])
        train_acc = accuracy(pred[train], labels[train])
        val_acc = accuracy(pred[val], labels[val])
        test_acc = accuracy(pred[test], labels[test])
        log.append([epoch, val_loss.item(), val_acc, test_acc])
        
        # set early stopping counter
        if val_acc > acc_best:
            best_epoch = epoch
            torch.save(net.state_dict(), args.dataset +'.pkl')
            no_improvement = 0
            acc_best = max(val_acc, acc_best)
            print('Saving model.....')
        else:
            no_improvement += 1
            if no_improvement == 300:
                print('Early stopping.')
                break

    dur.append(time.time() - t0)

    print("Epoch {:05d} | Train {:.4f} | Valid {:.4f} | Train {:.4f} | Val {:.4f} | Test {:.4f} | Time(s) {:.4f}".format(epoch, train_loss.item(), val_loss.item(), train_acc, val_acc, test_acc, np.mean(dur)))

''' Testing '''
net.load_state_dict(torch.load(args.dataset +'.pkl'))
net.eval()
test_logits,test_embeddings = net(g, feats)  
test_acc = torch.sum(test_logits[test].argmax(dim=1) == labels[test]).item() / len(test)

print("Test Acc: {:.4f}".format(test_acc))

pred = test_logits[test].cpu().detach().numpy().argmax(axis=1)
onehot = np.eye(dl.labels_train['num_classes'], dtype=np.int32)
pred = onehot[pred]

eval_res = dl.evaluate(pred)
print(eval_res['micro-f1'], eval_res['macro-f1'])

svm_macro_f1_list, svm_micro_f1_list, nmi_mean, nmi_std, ari_mean, ari_std = evaluate_results_nc(
            test_embeddings[test].cpu().detach().numpy(), dl.labels_test['data'][dl.labels_test['mask']].argmax(axis=1), num_classes=dl.labels_test['num_classes'])


loading data >>>>>>>>>>>>>>>>>>>>>
0 {'0': 3584}
1 {'0': 3584, '1': 5909}
building dgl.heterograph >>>>>>>>>>>>>>>>>>>>>
number_of_edges>>>>>>>> 0_edge1_1 ** 5957
number_of_edges>>>>>>>> 1_edge2_0 ** 5957
number_of_edges>>>>>>>> 1_edge3_1 ** 874
features_list0>>>>>>>> (3584, 300)
861 2508 215
nclass>>>>>>>>>>> 2
loss1，loss2>>>>>>>>>> tensor(0.5625, device='cuda:0', grad_fn=<NllLossBackward>) tensor(0.0020, device='cuda:0', grad_fn=<MeanBackward0>)
loss>>>>>>>>>> tensor(0.5644, device='cuda:0', grad_fn=<AddBackward0>)
Saving model.....
Epoch 00000 | Train 0.3021 | Valid 0.3015 | Train 0.5772 | Val 0.5953 | Test 0.5857 | Time(s) 0.2890
loss1，loss2>>>>>>>>>> tensor(0.5471, device='cuda:0', grad_fn=<NllLossBackward>) tensor(0.0015, device='cuda:0', grad_fn=<MeanBackward0>)
loss>>>>>>>>>> tensor(0.5486, device='cuda:0', grad_fn=<AddBackward0>)
Saving model.....
Epoch 00001 | Train 0.3006 | Valid 0.2999 | Train 0.5796 | Val 0.6093 | Test 0.5961 | Time(s) 0.2680
loss1，loss2>>>>>>>>>> tensor(0

In [1]:
1

1